In [1]:
!pip install utils

In [2]:
from scipy.stats import mannwhitneyu
import itertools
import numpy as np
from collections import defaultdict
from typing import Callable, Optional
import csv
import pandas as pd
import os
import utils

import utils


def is_valid_data_file(file_name:str) -> bool:
    return file_name.endswith("json") or file_name.endswith("txt")


def get_mean_for_combinations(df: pd.DataFrame, 
                       independent_variables: list[str], 
                       dependent_variables: list[str]) -> pd.DataFrame:

    # ensure all the columns are present in the df
    for col in independent_variables+dependent_variables:
        if col not in df:
            raise Exception(f"The column {col} is not in the dataframe\n\t(columns are {list(df.columns)})")
    assert(all(col in df for col in independent_variables))
    assert(dependent_variable in df for dependent_variable in dependent_variables)
    
    grouped = df.groupby(independent_variables, dropna=False)[dependent_variables].mean().reset_index()
    
    return grouped


import json
import os


def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Skip if 'results_by_tree' is not present
        if "results_by_tree" not in item:
            return []

        problem_name = item["problem_name"]
        pRef_method = item["pRef_method"]
        pRef_size = item["sample_size"]

        entries = item["results_by_tree"]

        def get_modified_entry(entry):
            entry["problem"] = problem_name
            entry["pRef_method"] = pRef_method
            entry["pRef_size"] = pRef_size

            errors = entry["results"]
            entry = entry | errors
            del entry["results"]

            if "order_tree" in entry:
                del entry["order_tree"]

            return entry

        entries = list(map(get_modified_entry, entries))
        return entries

    return [entry for item in data for entry in item_to_list_of_entries(item)]

'''
def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Handle both results_by_tree and tree_settings_list
        if "results_by_tree" in item:
            # Original logic for results data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            entries = item["results_by_tree"]
            
            def get_modified_entry(entry):
                entry["problem"] = problem_name
                entry["pRef_method"] = pRef_method
                entry["pRef_size"] = pRef_size
                errors = entry["results"]
                entry = entry | errors
                del entry["results"]
                if "order_tree" in entry:
                    del entry["order_tree"]
                return entry
            
            return list(map(get_modified_entry, entries))
            
        elif "tree_settings_list" in item:
            # New logic for configuration data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            has_error = 'error' in item
            error_msg = item.get('error', '')
            
            entries = []
            for tree_setting in item["tree_settings_list"]:
                for depth in tree_setting["depths"]:
                    entry = {
                        'problem': problem_name,
                        'pRef_method': pRef_method,
                        'pRef_size': pRef_size,
                        'kind': tree_setting['kind'],
                        'depth': depth,
                        'ps_budget': tree_setting['ps_budget'],
                        'ps_population': tree_setting['ps_population'],
                        'avoid_ancestors': tree_setting['avoid_ancestors'],
                        'metrics': tree_setting['metrics'],
                        'has_error': has_error,
                        'error_message': error_msg,
                        # No actual results available
                        'mse': None,
                        'mae': None,
                        'r_sq': None,
                        'evs': None
                    }
                    entries.append(entry)
            return entries
        else:
            return []

    return [entry for item in data for entry in item_to_list_of_entries(item)]


def json_to_entries(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Handle both results_by_tree and tree_settings_list
        if "results_by_tree" in item:
            # Original logic for results data
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            entries = item["results_by_tree"]
            
            def get_modified_entry(entry):
                entry["problem"] = problem_name
                entry["pRef_method"] = pRef_method
                entry["pRef_size"] = pRef_size
                errors = entry["results"]
                entry = entry | errors
                del entry["results"]
                if "order_tree" in entry:
                    del entry["order_tree"]
                return entry
            
            return list(map(get_modified_entry, entries))
            
        elif "tree_settings_list" in item:
            # FIXED: Handle different tree types properly
            problem_name = item["problem_name"]
            pRef_method = item["pRef_method"]
            pRef_size = item["sample_size"]
            has_error = 'error' in item
            error_msg = item.get('error', '')
            
            entries = []
            for tree_setting in item["tree_settings_list"]:
                for depth in tree_setting["depths"]:
                    # Base entry with common fields
                    entry = {
                        'problem': problem_name,
                        'pRef_method': pRef_method,
                        'pRef_size': pRef_size,
                        'kind': tree_setting['kind'],
                        'depth': depth,
                        'has_error': has_error,
                        'error_message': error_msg,
                        # No actual results available
                        'mse': None,
                        'mae': None,
                        'r_sq': None,
                        'evs': None
                    }
                    
                    # Add tree-type-specific fields
                    if tree_setting['kind'] == 'ps':
                        entry.update({
                            'ps_budget': tree_setting.get('ps_budget', None),
                            'ps_population': tree_setting.get('ps_population', None),
                            'avoid_ancestors': tree_setting.get('avoid_ancestors', None),
                            'metrics': tree_setting.get('metrics', None),
                            'cp': None  # PS doesn't have cp
                        })
                    elif tree_setting['kind'] == 'iai':
                        entry.update({
                            'ps_budget': None,  # IAI doesn't have these
                            'ps_population': None,
                            'avoid_ancestors': None,
                            'metrics': None,
                            'cp': tree_setting.get('cp', None)
                        })
                    elif tree_setting['kind'] == 'naive':
                        entry.update({
                            'ps_budget': None,  # Naive doesn't have these
                            'ps_population': None,
                            'avoid_ancestors': None,
                            'metrics': None,
                            'cp': None
                        })
                    
                    entries.append(entry)
            return entries
        else:
            return []

    return [entry for item in data for entry in item_to_list_of_entries(item)]
'''
def convert_accuracy_data_to_df(input_directory, output_filename):

    all_dicts = []
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Construct full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file is a JSON file
        if not os.path.isfile(file_path):
            continue

        if not is_valid_data_file(file_path):
            continue

        with open(file_path, 'r') as file:
            data = json.load(file)
            entries = json_to_entries(data)
            all_dicts.extend(entries)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_dicts)

    # Write the DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
   
def json_to_tree_data(data: dict):
    def item_to_list_of_entries(item) -> list[dict]:
        # Skip if 'results_by_tree' is not present
        if "results_by_tree" not in item:
            return []

        surrounding_information = {
            "problem": item["problem_name"],
            "pRef_method": item["pRef_method"]
        }

        entries = item["results_by_tree"]
        entries = [thing for thing in entries if "order_tree" in thing]

        def convert_order_tree(order_tree, accumulator=None, current_depth=0):
            if accumulator is None:
                accumulator = defaultdict(list)
            accumulator[current_depth].append(order_tree["own"])
            if len(order_tree["matching"]) > 0:
                convert_order_tree(order_tree["matching"], accumulator, current_depth + 1)
            if len(order_tree["unmatching"]) > 0:
                convert_order_tree(order_tree["unmatching"], accumulator, current_depth + 1)
            return accumulator

        def convert_tree_to_averages_by_level(entry):
            ps_search_info = {
                "ps_budget": entry["ps_budget"],
                "ps_population": entry["ps_population"],
                "metrics": entry["metrics"]
            }
            tree_structure = entry["order_tree"]
            just_depths = convert_order_tree(tree_structure)
            core_info_trees = [
                {"depth": depth, "order": order}
                for depth in just_depths
                for order in just_depths[depth]
            ]
            core_info_trees = [surrounding_information | ps_search_info | core_tree for core_tree in core_info_trees]
            return core_info_trees

        entries = list(map(convert_tree_to_averages_by_level, entries))
        return entries

    return [entry for item in data for entry in item_to_list_of_entries(item)]


def convert_tree_data_to_df(input_directory, output_filename):

    all_dicts = []
    # Iterate through all files in the input directory
    for filename in os.listdir(input_directory):
        # Construct full file path
        file_path = os.path.join(input_directory, filename)

        # Check if the file is a JSON file
        if not os.path.isfile(file_path):
            continue

        if not is_valid_data_file(file_path):
            continue


        with open(file_path, 'r') as file:
            data = json.load(file)
            entries = json_to_tree_data(data)
            all_dicts.extend(entries)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(all_dicts)

    # Write the DataFrame to a CSV file
    df.to_csv(output_filename, index=False)
    
    
def filter_dataframe(df, **kwargs):
    df = df.copy()  # Make a copy of the DataFrame to avoid modifying the original
    for col, value in kwargs.items():
        if col in df.columns:
            df = df[df[col] == value]
        else:
            raise ValueError(f"Column '{col}' not found in dataframe.")
    return df
        

    
    

def prettify_kind_column(df):
    kind_dict = {#"variance":"PS-W",
                 #"variance estimated_atomicity": "PS-WA",
                 "simplicity variance": "PS-SW",
                 "simplicity variance estimated_atomicity" :"PS-SWA"}
    
    df['kind'] = df.apply(
    lambda row: (
        kind_dict[row['metrics']] if row['kind'] == 'ps' else
        'Trad.' if row['kind'] == 'naive' else
        'IAI' if row['kind'] == 'iai' else
        row['kind']
    ),
    axis=1
)
    
    

    
    

In [3]:
import os

In [4]:
#run_location = r"/Users/gian/Desktop/CondorResults/VDT/compareown/run3/"
#run_location = r"C:\Users\gac8\Desktop\CondorResults\VDT\compareown\all_final_runs"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_07-29-H15'm'15's16"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-07-H16'm'47's09"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-10-H01'm'12's39"
#A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_07-29-H15'm'15's16

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-19-H16'm'26's53"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-19-H23'm'35's39"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\iai_run_3_08-20-H01'm'00's52"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-24-H02'm'25's36"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\iai_run_3_08-24-H11'm'06's57"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-24-H02'm'25's36"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-26-H14'm'06's32"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\compare_own_data_08-27-H12'm'29's47"

#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\all_final_runs"

#aco and pso
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\all_final_runs"

#bbo aco bro cro
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\ACO+BBO+BRO+CRO"

run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\all_final_runs_time_again"
#run_location = r"A:\metahuristic_benchmark\PS-descriptors\resources\variance_tree_materials\compare_own_data\all_output_files"
results_csv = os.path.join(run_location, "results.csv")
tree_data_csv = os.path.join(run_location, "tree_data.csv")

convert_accuracy_data_to_df(run_location, results_csv)
convert_tree_data_to_df(run_location, tree_data_csv)

#convert_accuracy_data_to_df(os.path.join(run_location, "data"), results_csv)
#convert_tree_data_to_df(os.path.join(run_location, "data"), tree_data_csv)



In [5]:

accuracy_data = pd.read_csv(results_csv)
prettify_kind_column(accuracy_data)
#tree_data = pd.read_csv(tree_data_csv)

display(accuracy_data)
#display(accuracy_data.dtypes)
#display(tree_data)


#prettify_kind_column(tree_data)


for kind in accuracy_data["kind"].unique():
    matching_rows = accuracy_data[accuracy_data['kind'] == kind]
    print(f"For the tree kind {kind}, there are {matching_rows.shape[0]} rows")

#headers = "kind,depth,ps_budget,ps_population,avoid_ancestors,metrics,problem,pRef_method,mse,mae,r_sq,evs"

,kind,depth,runtime_seconds,problem,pRef_method,pRef_size,mse,mae,r_sq,evs,ps_budget,ps_population,avoid_ancestors,metrics,cp
0,Trad.,3,1.004,SAT_S,CRO,10000,9.284650,2.400945,0.392605,0.399823,NaN,NaN,NaN,NaN,NaN
1,Trad.,4,1.004,SAT_S,CRO,10000,8.408765,2.304181,0.449905,0.451356,NaN,NaN,NaN,NaN,NaN
2,Trad.,5,1.004,SAT_S,CRO,10000,8.477953,2.199727,0.445378,0.446255,NaN,NaN,NaN,NaN,NaN
3,Trad.,3,2.264,SAT_S,BBO,10000,4.329518,1.745236,0.453288,0.469022,NaN,NaN,NaN,NaN,NaN
4,Trad.,4,2.264,SAT_S,BBO,10000,4.385156,1.679617,0.446262,0.446631,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,PS-SWA,3,NaN,SAT_S,PSO,10000,2.328190,1.168928,0.303515,0.327509,5000.0,100.0,False,simplicity variance estimated_atomicity,NaN
716,PS-SW,4,NaN,SAT_S,PSO,10000,3.149173,1.379821,0.321033,0.325723,5000.0,100.0,False,simplicity variance,NaN
717,PS-SWA,4,NaN,SAT_S,PSO,10000,4.512889,1.542640,0.237185,0.250058,5000.0,100.0,False,simplicity variance estimated_atomicity,NaN
718,PS-SW,5,NaN,SAT_S,PSO,10000,2.639485,1.240152,0.690232,0.706950,5000.0,100.0,False,simplicity variance,NaN


For the tree kind Trad., there are 450 rows
For the tree kind PS-SW, there are 90 rows
For the tree kind PS-SWA, there are 90 rows
For the tree kind IAI, there are 90 rows


In [6]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows


In [7]:

def generate_statistical_test_data(accuracy_data: pd.DataFrame, input_directory, output_filename):
    depths = [3, 4, 5]
    usable_data = filter_dataframe(accuracy_data, pRef_size=10000)
    usable_data = usable_data[usable_data["depth"].isin(depths)]
    
    result_column = "r_sq"
    
    # Check which methods are actually available in your data
    available_methods = usable_data["kind"].unique()
    print(f"Available methods in your data: {available_methods}")
    
    def winning_competitor_for_competition_and_values(problem: str, depth: int, metaheuristic: str):
        # Get data for all available methods
        method_data = {}
        for method in available_methods:
            data = filter_dataframe(usable_data, problem=problem, depth=depth, pRef_method=metaheuristic, kind=method)[result_column]
            if len(data) > 0:  # Only include methods with data
                method_data[method] = data
        
        # If we don't have enough methods to compare, return NaN
        if len(method_data) < 2:
            return {
                "problem": problem,
                "depth": depth,
                "metaheuristic": metaheuristic,
                "comparison": "insufficient_data",
                "p_value": float('nan'),
                "method_1": None,
                "method_2": None,
                "method_1_mean": float('nan'),
                "method_2_mean": float('nan')
            }
        
        # If we have PS-SW and PS-SWA, compare them
        if "PS-SW" in method_data and "PS-SWA" in method_data:
            p_value = mannwhitneyu(method_data["PS-SW"], method_data["PS-SWA"], alternative="two-sided").pvalue
            comparison = "PS-SW_vs_PS-SWA"
            method_1, method_2 = "PS-SW", "PS-SWA"
            method_1_mean = method_data["PS-SW"].mean()
            method_2_mean = method_data["PS-SWA"].mean()
        else:
            p_value = float('nan')
            comparison = "no_valid_comparison"
            method_1, method_2 = None, None
            method_1_mean, method_2_mean = float('nan'), float('nan')
        
        return {
            "problem": problem,
            "depth": depth,
            "metaheuristic": metaheuristic,
            "comparison": comparison,
            "p_value": p_value,
            "method_1": method_1,
            "method_2": method_2,
            "method_1_mean": method_1_mean,
            "method_2_mean": method_2_mean
        }
    
    all_problems = usable_data["problem"].unique()
    all_metaheuristics = usable_data



In [8]:
    
    
def generate_statistical_test_data(accuracy_data: pd.DataFrame, input_directory, output_filename):
    depths = [3, 4, 5]
    usable_data = filter_dataframe(accuracy_data, pRef_size = 10000)
    usable_data = usable_data[usable_data["depth"].isin(depths)]
    
    result_column = "r_sq"
    
    def winning_competitor_for_competition_and_values(problem: str, depth: int, metaheuristic: str) -> (str, np.ndarray):
        for_each_method = {tree_method: filter_dataframe(usable_data, problem = problem, depth = depth, pRef_method = metaheuristic, kind = tree_method)[result_column]
                           for tree_method in {"PS-SW", "PS-SWA", "IAI", "Trad."}}
        
        iai_average = np.average(for_each_method["IAI"])
        naive_average = np.average(for_each_method["Trad."])
        
        
        winning_competitor_method = "IAI" if iai_average > naive_average else "Trad."
        
        p_value_between_w_and_competitor = mannwhitneyu(for_each_method["PS-SW"], for_each_method[winning_competitor_method], alternative="greater").pvalue
        p_value_between_wa_and_competitor = mannwhitneyu(for_each_method["PS-SWA"], for_each_method[winning_competitor_method], alternative="greater").pvalue
        
        return {"problem": problem,
                "depth": depth,
                "metaheuristic": metaheuristic,
                "p_value_sw":p_value_between_w_and_competitor,
                "p_value_swa": p_value_between_wa_and_competitor,
                "winning_competitor": winning_competitor_method}
    
    
    
    all_problems = usable_data["problem"].unique()
    all_metaheuristics = usable_data["pRef_method"].unique()
    
    dicts = [winning_competitor_for_competition_and_values(problem=problem, depth = depth, metaheuristic=metaheuristic)
             for problem in all_problems
             for depth in depths
             for metaheuristic in all_metaheuristics]
    
    return pd.DataFrame(dicts)
        
        
    

statistical_data = generate_statistical_test_data(accuracy_data, None, None)

pivot_table = statistical_data.pivot_table(index=["problem", "depth", "metaheuristic"], 
                                            values =["p_value_sw", "p_value_swa"])
display(statistical_data)
display(pivot_table)

,problem,depth,metaheuristic,p_value_sw,p_value_swa,winning_competitor
0,SAT_S,3,CRO,0.166667,0.500000,Trad.
1,SAT_S,3,BBO,0.500000,1.000000,IAI
2,SAT_S,3,ACO,0.166667,0.333333,Trad.
3,SAT_S,3,BRO,0.166667,1.000000,Trad.
4,SAT_S,3,PSO,0.500000,0.833333,Trad.
5,SAT_S,4,CRO,0.333333,0.500000,Trad.
6,SAT_S,4,BBO,1.000000,0.500000,IAI
7,SAT_S,4,ACO,0.833333,0.166667,Trad.
8,SAT_S,4,BRO,0.666667,1.000000,Trad.
9,SAT_S,4,PSO,0.833333,1.000000,Trad.


p_value_sw  p_value_swa
problem depth metaheuristic                         
BT      3     ACO              0.166667     0.333333
              BBO              0.166667     0.333333
              BRO              0.500000     0.833333
              CRO              0.666667     0.833333
              PSO              0.500000     1.000000
        4     ACO              0.500000     1.000000
              BBO              0.166667     0.166667
              BRO              0.666667     1.000000
              CRO              0.166667     0.666667
              PSO              1.000000     1.000000
        5     ACO              1.000000     1.000000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              0.166667     0.500000
              PSO              1.000000     0.500000
GC_L    3     ACO              1.000000     0.500000
              BBO              0.500000     0.500000
              BRO              1.000000     0.833333
              CRO              0.166667     0.666667
              PSO              0.500000     0.500000
        4     ACO              0.500000     1.000000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              0.500000     0.500000
              PSO              0.500000     0.500000
        5     ACO              1.000000     1.000000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              0.500000     0.500000
              PSO              1.000000     0.500000
GC_S    3     ACO              0.500000     0.500000
              BBO              0.166667     0.166667
              BRO              0.666667     1.000000
              CRO              0.166667     0.333333
              PSO              0.166667     0.166667
        4     ACO              1.000000     1.000000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              0.166667     0.166667
              PSO              0.500000     1.000000
        5     ACO              1.000000     0.500000
              BBO              0.500000     0.500000
              BRO              1.000000     1.000000
              CRO              1.000000     0.500000
              PSO              1.000000     0.500000
SAT_L   3     ACO              0.666667     0.166667
              BBO              0.166667     0.166667
              BRO              0.500000     0.500000
              CRO              0.166667     0.166667
              PSO              0.166667     0.166667
        4     ACO              0.166667     0.500000
              BBO              0.500000     0.500000
              BRO              0.500000     0.500000
              CRO              0.166667     0.500000
              PSO              0.500000     0.500000
        5     ACO              0.166667     0.166667
              BBO              0.500000     0.500000
              BRO              1.000000     0.500000
              CRO              0.166667     0.500000
              PSO              0.500000     0.500000
SAT_M   3     ACO              0.166667     0.166667
              BBO              0.166667     0.166667
              BRO              0.166667     1.000000
              CRO              0.500000     0.500000
              PSO              0.166667     0.166667
        4     ACO              0.500000     0.500000
              BBO              0.666667     0.166667
              BRO              0.833333     0.333333
              CRO              0.500000     1.000000
              PSO              0.166667     0.166667
        5     ACO              0.166667     0.166667
              BBO              0.166667     0.666667
              BRO              0.166667     0.666667
              CRO              0.500000     1.000000
 

In [9]:
statistical_data = generate_statistical_test_data(accuracy_data, None, None)


In [10]:
'''
def bold_max(row):
    row_as_numbers = [float(item[:-1]) for item in row]
    max_number = max(row_as_numbers)
    
    return ['font-weight: bold' if item == max_number else '' for item in row_as_numbers]
'''


def bold_max(row):
    row_as_numbers = []
    for item in row:
        if isinstance(item, str) and item.endswith("%"):
            # Remove % and convert string to float
            row_as_numbers.append(float(item[:-1]))
        else:
            # Already a number (int or float)
            row_as_numbers.append(float(item))
    max_number = max(row_as_numbers)
    return ['font-weight: bold' if num == max_number else '' for num in row_as_numbers]


def style_pivot_table(pivot_table):
    custom_column_order = ['PS-SW', 'PS-SWA', 'IAI', 'Trad.']

    pivot_table = pivot_table.mul(100).round(1).astype(str) + "%"    

    # Reorder columns based on custom order
    pivot_table = pivot_table.reindex(columns=custom_column_order)

    styled_df = pivot_table.style.apply(bold_max, axis=1)

    return styled_df

def put_latex_tables_side_by_side(left_latex, right_latex):
    return r"\begin{tabular}{ccccccc}\hline"+left_latex+r"\\ \hline\end{tabular}\quad\begin{tabular}{ccccccc}\hline"+right_latex+r"\\ \hline\end{tabular}"

def fix_latex(input_string):
    # Replace '%' with '\%'
    replacements = {"%":"\\%",
                    "pRef_method":"Met.",
                    "{SA}": r"{\rotcell{SA}}", # note that SA is a subset of SAT\_50 etc.., so it causes some issues
                    "SAT_S": "SAT\_20",
                    "SAT_M": "SAT\_50",
                    "SAT_L": "SAT\_100",
                    "GC_L": "GC\_anna",
                    "GC_S": "GC\_jean",
                    "uniform": "RS",
                    "kind": "tree",
                    r"\multirow[c]{12}" : r"\hline \multirow[c]{12}",
                    r"& \multirow[c]{3}" : r"\cline{2-7} & \multirow[c]{3}",
                    r"} \cline{2-7}" : "} ",
                   "\\font-weightbold": "",
                    "≪": "\ll "}

    texts_to_rotate = ["problem", "BT", "GC\_anna", "GC\_jean", "SAT\_20",  "SAT\_50",  "SAT\_100", "Met.", "GA", "Tabu", "RS", "PSO", "depth"]  # MOD: added PSO to rotate label in LaTeX output

    for item_to_rotate in texts_to_rotate:
        replacements[item_to_rotate] = r"\rotcell{"+item_to_rotate+"}"

    modified_string = str(input_string)
    for orig, replacement in replacements.items():
        modified_string = modified_string.replace(orig, replacement)
    
    return modified_string


def pivot_table_as_latex(pivot_table):
    latex_text = pivot_table.to_latex(convert_css=True)
    latex_text = fix_latex(latex_text)
    return latex_text

In [11]:
pRef_size = 10000
depths = [3, 4, 5]


usable_data = accuracy_data.copy()
usable_data = usable_data[usable_data["pRef_size"] == pRef_size] 
usable_data = usable_data[usable_data["depth"].isin(depths)]

independent_variables = ["problem", "pRef_method", "kind", "depth"]
dependent_variables = ["r_sq"]


problems = [ "GC_S", "GC_L", "SAT_S", "SAT_M", "SAT_L", "BT"]

left_problems, right_problems = problems[:3], problems[3:]

def make_table_for_problems(problem_subset):
    with_right_problems = usable_data[usable_data['problem'].isin(problem_subset)]
    pivot_table = with_right_problems.pivot_table(index = ["problem", "pRef_method", "depth"], 
                                        columns = ["kind"], 
                                        values = dependent_variables[0])
    return style_pivot_table(pivot_table)



left_table = make_table_for_problems(left_problems)
right_table = make_table_for_problems(right_problems)

display(left_table)
display(right_table)

left_table_latex = pivot_table_as_latex(left_table)
right_table_latex = pivot_table_as_latex(right_table)

full_table_latex = put_latex_tables_side_by_side(left_table_latex, right_table_latex)

print("left table:")
print(left_table_latex)

print("\n\n\n\n\n\nright table")
print(right_table_latex)



# pivot_table = usable_data.pivot_table(index = ["problem", "pRef_method", "depth"], 
#                                         columns = ["kind"], 
#                                         values = dependent_variables[0])





# for problem in usable_data['problem'].unique():
#     with_right_problem = usable_data[usable_data['problem'] == problem]
#     pivot_table = with_right_problem.pivot_table(index = ["pRef_method", "depth"], 
#                                         columns = ["kind"], 
#                                         values = dependent_variables[0])

#     pivot_table = style_pivot_table(pivot_table)
#     print(fix_latex(f"{problem = }"))
#     print_pivot_table_as_latex(pivot_table)
    
    # Display the styled dataframe
    #display(styled_df)
    
    #display(pivot_table)


left table:
\begin{tabular}{lllllll}
 &  & tree & PS-SW & PS-SWA & IAI & Trad. \\
\rotcell{problem} & \rotcell{Met.} & \rotcell{depth} &  &  &  &  \\
\multirow[c]{15}{*}{\rotcell{GC\_anna}}  & \multirow[c]{3}{*}{ACO} & 3 & 31.5\% & \bfseries 48.6\% & 33.3\% & 31.1\% \\
 &  & 4 & \bfseries 39.9\% & 30.8\% & 33.3\% & 29.5\% \\
 &  & 5 & 13.6\% & 22.1\% & \bfseries 33.3\% & 21.5\% \\
 \cline{2-7} & \multirow[c]{3}{*}{BBO} & 3 & \bfseries 85.0\% & 80.0\% & 75.9\% & 70.8\% \\
 &  & 4 & \bfseries 90.2\% & 85.7\% & 76.4\% & 75.7\% \\
 &  & 5 & \bfseries 91.8\% & 81.1\% & 81.0\% & 77.9\% \\
 \cline{2-7} & \multirow[c]{3}{*}{BRO} & 3 & -0.5\% & 52.8\% & 45.2\% & \bfseries 57.2\% \\
 &  & 4 & 49.2\% & 57.1\% & 54.9\% & \bfseries 64.3\% \\
 &  & 5 & 51.6\% & 59.3\% & 62.4\% & \bfseries 68.7\% \\
 \cline{2-7} & \multirow[c]{3}{*}{CRO} & 3 & \bfseries 86.0\% & 70.6\% & 42.9\% & 68.0\% \\
 &  & 4 & \bfseries 86.9\% & 84.4\% & 78.6\% & 70.5\% \\
 &  & 5 & 83.9\% & \bfseries 85.9\% & 78.5\% & 69.2\% \

In [12]:
result = []
print(accuracy_data["problem"].unique())
print(accuracy_data["pRef_method"].unique())
for method in accuracy_data["pRef_method"].unique():
    for problem in accuracy_data["problem"].unique():
        for depth in [3, 4, 5]:
            appropriate_data = filter_dataframe(accuracy_data, depth = depth, pRef_method = method, problem = problem, pRef_size=10000)
            trad_data = appropriate_data[appropriate_data["kind"] == "Trad."]["r_sq"][:100]
            own_data = appropriate_data[appropriate_data["kind"] == "PS-SW"]["r_sq"][:100]
            p_value = mannwhitneyu(x = own_data, y = trad_data, alternative="greater").pvalue
            diff = np.average(own_data)-np.average(trad_data)
            
            p_value_string = "0.001" if p_value < 0.001 else round(p_value, 3)
            res = f"{round(diff*100, 2)}%, {p_value_string}"
            if p_value < 0.05:
                res = "\\bfseries "+res
            
            
            result.append({"method":method,
                           "problem":problem,
                           "depth":depth,
                           "res":res})
            
        
improvements = pd.DataFrame(result)
display(improvements)
pivot_table = improvements.pivot_table(index=["problem", "method"],
                                       columns="depth",
                                       values=["res"],
                                       aggfunc=lambda x:x)

#pivot_table = pivot_table.mul(100).round(1).astype(str) + "%"    

display(pivot_table)

latex_code = pivot_table.to_latex()

def fix_latex(input_string):
    # Replace '%' with '\%'
    replacements = {"%":"\\%",
                    "pRef_method":"Met.",
                    "BT":"Staff R.",
                    "SAT_S": "SAT\_20",
                    "SAT_M": "SAT\_50",
                    "SAT_L": "SAT\_100",
                    "GC_L": "GC\_anna",
                    "GC_S": "GC\_jean",
                    "uniform": "RS",
                    "kind": "tree",
                   "\\font-weightbold": "",
                    "res": "average R^2 improvemennt between PS-SW and Trad, with p-value",
                    "≪": "\ll "}


    modified_string = str(input_string)
    for orig, replacement in replacements.items():
        modified_string = modified_string.replace(orig, replacement)
    
    return modified_string
latex_code = fix_latex(latex_code)
print(latex_code)


['SAT_S' 'SAT_M' 'SAT_L' 'GC_S' 'GC_L' 'BT']
['CRO' 'BBO' 'ACO' 'BRO' 'PSO']


,method,problem,depth,res
0,CRO,SAT_S,3,"17.0%, 0.167"
1,CRO,SAT_S,4,"24.79%, 0.333"
2,CRO,SAT_S,5,"29.1%, 0.333"
3,CRO,SAT_M,3,"9.61%, 0.5"
4,CRO,SAT_M,4,"22.68%, 0.333"
5,CRO,SAT_M,5,"23.95%, 0.167"
6,CRO,SAT_L,3,"7.33%, 0.167"
7,CRO,SAT_L,4,"8.73%, 0.167"
8,CRO,SAT_L,5,"7.5%, 0.167"
9,CRO,GC_S,3,"10.85%, 0.167"


res                               
depth                       3               4              5
problem method                                              
BT      ACO      7.99%, 0.167    9.42%, 0.167     1.66%, 0.5
        BBO      8.04%, 0.167    6.41%, 0.167   4.27%, 0.333
        BRO       -0.87%, 0.5   -2.91%, 0.667   -10.65%, 1.0
        CRO     -0.24%, 0.667     4.0%, 0.167   5.53%, 0.167
        PSO     15.69%, 0.167      5.48%, 0.5   6.71%, 0.333
GC_L    ACO      0.39%, 0.667     10.44%, 0.5  -7.94%, 0.667
        BBO     14.24%, 0.167   14.52%, 0.167  13.91%, 0.167
        BRO      -57.65%, 1.0    -15.06%, 1.0   -17.18%, 1.0
        CRO     18.01%, 0.167   16.41%, 0.167  14.67%, 0.333
        PSO     18.27%, 0.167   15.44%, 0.167   -29.74%, 1.0
GC_S    ACO        4.91%, 0.5    -18.09%, 1.0     2.12%, 0.5
        BBO     18.64%, 0.167    9.62%, 0.167   8.74%, 0.167
        BRO     -1.45%, 0.667     -17.5%, 1.0   -17.97%, 1.0
        CRO     10.85%, 0.167    13.3%, 0.167    -0.36%, 0.5
        PSO      12.6%, 0.167    8.85%, 0.167  -1.23%, 0.833
SAT_L   ACO      3.35%, 0.667   10.25%, 0.167   9.16%, 0.167
        BBO     20.12%, 0.167   17.95%, 0.167  17.38%, 0.167
        BRO     17.11%, 0.167   19.98%, 0.167   0.08%, 0.667
        CRO      7.33%, 0.167    8.73%, 0.167    7.5%, 0.167
        PSO     20.78%, 0.167   19.85%, 0.333    10.34%, 0.5
SAT_M   ACO     14.18%, 0.167    9.73%, 0.167   5.45%, 0.167
        BBO      23.3%, 0.167   10.27%, 0.667  19.85%, 0.167
        BRO     15.11%, 0.167   -6.66%, 0.833  12.62%, 0.167
        CRO        9.61%, 0.5   22.68%, 0.333  23.95%, 0.167
        PSO     35.98%, 0.167    23.2%, 0.167  21.88%, 0.167
SAT_S   ACO     14.76%, 0.167  -16.47%, 0.833  -0.22%, 0.667
        BBO     17.45%, 0.333   -1.01%, 0.667  34.72%, 0.167
        BRO     13.32%, 0.167   -0.85%, 0.667  -4.11%, 0.833
        CRO      17.0%, 0.167   24.79%, 0.333   29.1%, 0.333
        PSO        1.53%, 0.5  -15.49%, 0.833  17.67%, 0.167

\begin{tabular}{lllll}
\toprule
 &  & \multicolumn{3}{r}{average R^2 improvemennt between PS-SW and Trad, with p-value} \\
 & depth & 3 & 4 & 5 \\
problem & method &  &  &  \\
\midrule
\multirow[t]{5}{*}{Staff R.} & ACO & 7.99\%, 0.167 & 9.42\%, 0.167 & 1.66\%, 0.5 \\
 & BBO & 8.04\%, 0.167 & 6.41\%, 0.167 & 4.27\%, 0.333 \\
 & BRO & -0.87\%, 0.5 & -2.91\%, 0.667 & -10.65\%, 1.0 \\
 & CRO & -0.24\%, 0.667 & 4.0\%, 0.167 & 5.53\%, 0.167 \\
 & PSO & 15.69\%, 0.167 & 5.48\%, 0.5 & 6.71\%, 0.333 \\
\cline{1-5}
\multirow[t]{5}{*}{GC\_anna} & ACO & 0.39\%, 0.667 & 10.44\%, 0.5 & -7.94\%, 0.667 \\
 & BBO & 14.24\%, 0.167 & 14.52\%, 0.167 & 13.91\%, 0.167 \\
 & BRO & -57.65\%, 1.0 & -15.06\%, 1.0 & -17.18\%, 1.0 \\
 & CRO & 18.01\%, 0.167 & 16.41\%, 0.167 & 14.67\%, 0.333 \\
 & PSO & 18.27\%, 0.167 & 15.44\%, 0.167 & -29.74\%, 1.0 \\
\cline{1-5}
\multirow[t]{5}{*}{GC\_jean} & ACO & 4.91\%, 0.5 & -18.09\%, 1.0 & 2.12\%, 0.5 \\
 & BBO & 18.64\%, 0.167 & 9.62\%, 0.167 & 8.74\%, 0.167 \\
 & BRO & 